In [ ]:
pip install keras tensorflow pillow

In [7]:
import numpy as np
import os
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image

In [13]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3), activation='relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))

# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Part 2 - Fitting the CNN to the images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(r'C:\archive\casting_data\casting_data\train',
                                                 target_size=(300, 300),
                                                 batch_size=32,
                                                 class_mode='binary')

test_set = test_datagen.flow_from_directory(r'C:\archive\casting_data\casting_data\test',
                                             target_size=(300, 300),
                                             batch_size=32,
                                             class_mode='binary')

model = classifier.fit(training_set,
                        steps_per_epoch=1000,
                        epochs=100,
                        validation_data=test_set,
                        validation_steps=1000)

# Save the model
classifier.save("vggmodel.h5")
print("Saved model to disk")

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


C:\Users\roxna\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
 208/1000 ━━━━━━━━━━━━━━━━━━━━ 24:42 2s/step - accuracy: 0.5668 - loss: 9.5090

C:\anaconda\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
C:\Users\roxna\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 397s 393ms/step - accuracy: 0.6221 - loss: 4.1904 - val_accuracy: 0.7315 - val_loss: 0.4978
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 367s 366ms/step - accuracy: 0.7683 - loss: 0.4771 - val_accuracy: 0.8462 - val_loss: 0.3755
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 371s 369ms/step - accuracy: 0.8434 - loss: 0.3646 - val_accuracy: 0.8699 - val_loss: 0.2959
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 386s 384ms/step - accuracy: 0.8752 - loss: 0.3020 - val_accuracy: 0.9385 - val_loss: 0.2136
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 287s 286ms/step - accuracy: 0.8994 - loss: 0.2602 - val_accuracy: 0.9594 - val_loss: 0.1594
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 325s 324ms/step - accuracy: 0.9230 - loss: 0.2092 - val_accuracy: 0.7846 - val_loss: 0.4183
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 324s 322ms/step - accuracy: 0.8884 - loss: 0.2596 - val_accuracy: 0.9678 - val_loss: 0.1361
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 325s 323ms/step - accura

Saved model to disk


In [15]:
# Import necessary libraries
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

# Load the model
model = load_model('vggmodel.h5')
print('Model Loaded Successfully')

# Function to predict an image
def predict_image(image_path):
    test_image = load_img(image_path, target_size=(300, 300))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)
    
    # Assuming a binary classification with a threshold of 0.5
    if result[0][0] >= 0.5:
        return 'CASTING IS NOT DEFECTIVE'
    else:
        return 'CASTING IS DEFECTIVE'

Model Loaded Successfully


In [8]:
## Step 6: Build the GUI
# Create a window 
from tkinter import Tk, Label, Button, filedialog, OUTSIDE
from PIL import Image, ImageTk
root = Tk()   
root.title("CASTING INSPECTOR")   
root.geometry("700x450")   
root.resizable(width=False, height=False) 

# Load the model
model = load_model('vggmodel.h5')
print('Model Loaded Successfully')

# Function to open image
def open_img(): 
    x = openfilename() 
    img = Image.open(x) 
    img.save("casting.jpeg")
    img = ImageTk.PhotoImage(img) 
    panel = Label(root, image=img)   
    panel.image = img 
    panel.place(bordermode=OUTSIDE, x=50, y=50)

# Function to open file dialog
def openfilename(): 
    filename = filedialog.askopenfilename(title='Select Image') 
    return filename

# Function for prediction
def prediction():
    img = "casting.jpeg"    
    test_image = load_img(img, target_size=(300, 300))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)
    
    if result[0][0] == 1:
        prediction_text = 'CASTING IS NOT DEFECTIVE'
    else:
        prediction_text = 'CASTING IS DEFECTIVE'
        
    result_label = Label(root, text=prediction_text)   
    result_label.place(bordermode=OUTSIDE, x=400, y=120)

# Create buttons and labels
btn_open_image = Button(root, text='Open Image', command=open_img)
btn_open_image.place(x=10, y=400)

btn_predict = Button(root, text='Predict', command=prediction)
btn_predict.place(x=200, y=400)

result_hd = Label(root, text="RESULT")
result_hd.place(bordermode=OUTSIDE, x=400, y=90)

# Start the GUI loop
root.mainloop()

NameError: name 'load_model' is not defined